Straight forward method to extract relevant details using string pattern matching
Its easy to implement, but need proper string pattern mapping for this to work thus its less robust solution for the problem.

In [110]:
import re
from pdfminer.high_level import extract_pages, extract_text
import pdfplumber
import numpy as np
import pytesseract
from PIL import Image
import pandas as pd


def extract_details_from_text(text):
    invoice_details = {}

    # Use regular expressions to find the relevant information
    account_pattern = r"Account #:\s*(\d+)"
    ifsc_pattern = r"IFSC Code:\s*([A-Z0-9]+)"
    bank_pattern = r"Bank:\s*([A-Za-z\s]+)"
    branch_pattern = r"Branch:\s*([A-Za-z\s-]+)"

    # Search for the patterns in the extracted text
    account_match = re.search(account_pattern, text)
    ifsc_match = re.search(ifsc_pattern, text)
    bank_match = re.search(bank_pattern, text)
    branch_match = re.search(branch_pattern, text)

    # Store the matched groups in the dictionary
    if account_match:
        invoice_details['Account'] = account_match.group(1)
    if ifsc_match:
        invoice_details['IFSC'] = ifsc_match.group(1)
    if bank_match:
        invoice_details['Bank'] = bank_match.group(1).split("\n")[0]
    if branch_match:
        invoice_details['Branch'] = branch_match.group(1).split("\n")[0]


    gstin_pattern = r"GSTIN\s*([\dA-Z]+)"
    address_pattern = r"C/o\s*([\w\s,.-]+)"
    mobile_pattern = r"Mobile\s*\+?(\d{2}\s*\d{10})"
    email_pattern = r"Email\s*([\w\.-]+@[\w\.-]+)"
    invoice_number_pattern = r"Invoice #:\s*(\w+-\d+)"
    invoice_date_pattern = r"Invoice Date:\s*([\d\s\w]+)"
    due_date_pattern = r"Due Date:\s*([\d\s\w]+)"
    customer_pattern = r"Customer Details:\s*([\w\s]+)"
    place_of_supply_pattern = r"Place of Supply:\s*([\dA-Z\s-]+)"
    

    # Search for the patterns in the extracted text
    gstin_match = re.search(gstin_pattern, text)
    address_match = re.search(address_pattern, text)
    mobile_match = re.search(mobile_pattern, text)
    email_match = re.search(email_pattern, text)
    invoice_number_match = re.search(invoice_number_pattern, text)
    invoice_date_match = re.search(invoice_date_pattern, text)
    due_date_match = re.search(due_date_pattern, text)
    customer_match = re.search(customer_pattern, text)
    place_of_supply_match = re.search(place_of_supply_pattern, text)

    # Store the matched groups in the dictionary
    if gstin_match:
        invoice_details['GSTIN'] = gstin_match.group(1)
    if address_match:
        invoice_details['Address'] = ",".join(address_match.group(1).strip().split("\n")[:-1])
    if mobile_match:
        invoice_details['Mobile'] = mobile_match.group(1).split()[1]
    if email_match:
        invoice_details['Email'] = email_match.group(1)
    if invoice_number_match:
        invoice_details['Invoice Number'] = invoice_number_match.group(1)
    if invoice_date_match:
        invoice_details['Invoice Date'] = invoice_date_match.group(1).split("\n")[0]
    if due_date_match:
        invoice_details['Due Date'] = due_date_match.group(1).split("\n")[0]
    if customer_match:
        invoice_details['Customer'] = customer_match.group(1).strip().split("\n")[0]
    if place_of_supply_match:
        invoice_details['Place of Supply'] = place_of_supply_match.group(1).strip()
    return invoice_details

In [111]:
def extract_text_from_pdf(pdf_path):
    try:
        # Try extracting text using pdfminer for regular PDFs
        text = extract_text(pdf_path)
        if text.strip():
            return text, "regular"
        else:
            return None, None
    except Exception as e:
        print(f"Error in pdfminer extraction: {e}")
        return None, None
    
def convert_pdf_to_images(pdf_path):
    # Convert PDF to images for OCR
    from pdf2image import convert_from_path
    return convert_from_path(pdf_path)

def extract_text_from_image_pdf(pdf_path):
    try:
        # Convert PDF to image using OpenCV and extract text using Tesseract
        images = convert_pdf_to_images(pdf_path)
        extracted_text = ''
        for img in images:
            text = pytesseract.image_to_string(img)
            extracted_text += text
        return extracted_text, "scanned"
    except Exception as e:
        print(f"Error in OCR extraction: {e}")
        return None, None


In [112]:

def process_invoice(pdf_path):
    # Step 1: Try extracting as a regular PDF
    text, doc_type = extract_text_from_pdf(pdf_path)
    
    if text is None:
        # Step 2: If not a regular PDF, try OCR
        text, doc_type = extract_text_from_image_pdf(pdf_path)
        

    if text:
        # Step 3: Extract invoice data from the text
        extracted_data = extract_details_from_text(text)

        return {
            'document_type': doc_type,
            'extracted_data': extracted_data,
        }
    else:
        return {
            'document_type': None,
            'error': 'Unable to extract text'
        }

    

In [113]:
data = process_invoice('sample.pdf')
print(data)

{'document_type': 'regular', 'extracted_data': {'Account': '1146860541', 'Bank': 'Kotak Mahindra Bank', 'Branch': 'PUNE - CHINCHWAD', 'GSTIN': '23AADCU2395N1ZY', 'Address': 'KARUNA GUPTA KURELE, 1st Floor,S.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol,Shahdol, MADHYA PRADESH, 484001', 'Mobile': '8585960963', 'Email': 'ruhi@dermaq.in', 'Invoice Number': 'INV-73', 'Invoice Date': '15 Feb 2024', 'Due Date': '26 Oct 2023', 'Customer': 'Avik Mallick', 'Place of Supply': '23-MADHYA PRADESH'}}
